In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import cv2
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src.datasets import get_dataset, get_dataloader
from src.utils import parse_data_cfg, IMG, DATA_DIR, RHD

In [ ]:
cfg = parse_data_cfg(WORK_DIR/'data_cfg'/'exp2/znb_pose_data.cfg')
epoch = 200
exp_dir = cfg["exp_dir"]
data_split = 'train'
split_set = cfg[data_split + '_set']
img_size = int(cfg['img_size'])

In [ ]:
xyz_gt_list, uv_gt_list, vis_list, K_list, _ = RHD.get_all_anno(split_set)

# Visualize Dataloader

In [ ]:
dataset_kwargs = {'split_set': split_set}
dataset   = get_dataset(cfg, dataset_kwargs)
sampler   = None
shuffle   = cfg['shuffle']
kwargs = {'batch_size'  :   int(cfg['batch_size']),
          'shuffle'     :   shuffle,
          'num_workers' :   int(cfg['num_workers']),
          'pin_memory'  :   True}
data_loader = get_dataloader(dataset, sampler, kwargs)


In [ ]:
idx = 0
for i, (img, scoremap, uv_gt, vis) in enumerate(data_loader):
    if i == idx:
        batch_size = img.shape[0]
        img = img.cpu().numpy()
        img = np.swapaxes(img, 2, 3)
        img = np.swapaxes(img, 1, 3)
        img = IMG.scale_img_255(img)
        scoremap = scoremap.cpu().numpy()
        scoremap = np.swapaxes(scoremap, 2, 3)
        scoremap = np.swapaxes(scoremap, 1, 3)
        uv_gt = uv_gt.cpu().numpy()
        vis = vis.cpu().numpy()
        break
    i += 1

In [ ]:
idx = 6
cur_smap = scoremap[idx]
cur_img = img[idx]
cur_uv = uv_gt[idx]
cur_vis = vis[idx].astype('bool')

fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i in range(5):
    for j in range(5):
        k = np.ravel_multi_index((i, j), (5, 5))
        if k >= cur_smap.shape[2]:
            break        
        show_scoremap = cv2.cvtColor(cur_smap[:, :, k], cv2.COLOR_GRAY2RGB)
        show_scoremap = cv2.normalize(show_scoremap, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
        show_scoremap[:, :, 0] = 0
        show_scoremap[:, :, 2] = 0
        show = 0.5*show_scoremap + 0.5*cur_img
    
        ax[i, j].imshow(show.astype('uint32'))
        ax[i, j].plot(cur_uv[k, 0], cur_uv[k, 1], 'r.')
        RHD.visualize_joints_2d(ax[i, j], cur_uv[RHD.REORDER_IDX])

fig, ax = plt.subplots()
ax.imshow(cur_img)
RHD.visualize_joints_2d(ax, cur_uv[RHD.REORDER_IDX])

# Evaluation

In [ ]:
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_uv.txt'.format(epoch, data_split))
pred_uv = np.reshape(np.loadtxt(pred_file), (-1, 21, 2))

revert_pred_uv = []
for i, uv in enumerate(pred_uv):
    img = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', split_set, 
                                             'color', '%.5d.png' % i)))
    revert_pred_uv.append(RHD.revert_hand_annot(img, uv_gt_list[i], uv, (img_size, img_size)))
revert_pred_uv = np.asarray(revert_pred_uv)

In [ ]:
uv_gt_list = np.asarray(uv_gt_list)

error, uv_error = RHD.mean_L2_error_vis(uv_gt_list[..., :2], revert_pred_uv, vis_list)
min_error_idx = np.argmin(error)
max_error_idx = np.argmax(error)

print('%s UV mean_l2_error: ' %data_split, uv_error)
print('Best Pose id:', min_error_idx, 'uv_error:', error[min_error_idx])
print('Worst Pose id:', max_error_idx, 'uv_error:', error[max_error_idx])
for idx in np.argsort(error):
    print(idx, error[idx])

In [ ]:
pck = RHD.percentage_frames_within_error_curve_vis(uv_gt_list, revert_pred_uv, vis_list)
print(pck)
thresholds = np.arange(0, 85, 5)
print('AUC:', RHD.calc_auc(pck, thresholds))

In [ ]:
idx = 1012
image = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', split_set, 
                                             'color', '%.5d.png' % idx)))

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image)
cur_vis = vis_list[idx]
# RHD.visualize_joints_2d(ax, revert_pred_uv[idx][RHD.REORDER_IDX], c='r')
# RHD.visualize_joints_2d(ax, uv_gt_list[idx][RHD.REORDER_IDX], c='b')
ax.plot(uv_gt_list[idx][cur_vis, 0], uv_gt_list[idx][cur_vis, 1], 'bo')
ax.plot(revert_pred_uv[idx][cur_vis, 0], revert_pred_uv[idx][cur_vis, 1], 'ro')

# Model detection

In [ ]:
from src.models import get_model
import torch
torch.cuda.set_device(1)
model = get_model(cfg, False, epoch, None, None)
img, scoremap, uv_gt, vis = next(iter(data_loader))
img = img[0].unsqueeze(0)
pred_smap, pred_uv = model.detect(img.cuda())
img = img.cpu().numpy()

In [ ]:
img = np.swapaxes(img, 2, 3)
img = np.swapaxes(img, 1, 3)
img = IMG.scale_img_255(img[0])
fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i in range(5):
    for j in range(5):
        k = np.ravel_multi_index((i, j), (5, 5))
        if k >= pred_smap.shape[2]:
            break        
        show_scoremap = cv2.cvtColor(pred_smap[:, :, k], cv2.COLOR_GRAY2RGB)
        show_scoremap = cv2.normalize(show_scoremap, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
        show_scoremap[:, :, 0] = 0
        show_scoremap[:, :, 2] = 0
        show = 0.5*show_scoremap + 0.5*img
    
        ax[i, j].imshow(show.astype('uint32'))
        ax[i, j].plot(pred_uv[k, 0], pred_uv[k, 1], 'r.')
#         RHD.visualize_joints_2d(ax[i, j], cur_uv[RHD.REORDER_IDX])